In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set random seeds for reproducibility
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


data = pd.read_csv("/content/heart_.csv")
data.head()

le_model = LabelEncoder()

data['Sex'] = le_model.fit_transform(data['Sex'])
data['ChestPainType'] = le_model.fit_transform(data['ChestPainType'])
data['RestingECG'] = le_model.fit_transform(data['RestingECG'])
data['ExerciseAngina'] = le_model.fit_transform(data['ExerciseAngina'])
data['ST_Slope'] = le_model.fit_transform(data['ST_Slope'])

X=data.drop('HeartDisease', axis=1)
y=data['HeartDisease']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

# Standardize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build your DNN model
num_features = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(num_features,)),
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define an initial random state to be used in the model initialization
initial_random_state = np.random.get_state()

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Restore the random state after model training to ensure reproducibility
np.random.set_state(initial_random_state)
tf.random.set_seed(seed_value)

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)

from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import roc_auc_score

y_pred_prob_DNN = model.predict(X_test).ravel()
auc_roc_DNN = roc_auc_score(y_test, y_pred_prob_DNN)
print(f'DNN AUC: {auc_roc_DNN}')

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))



Epoch 1/100
24/24 [==============================] - 1s 11ms/step - loss: 0.6911 - accuracy: 0.5908 - val_loss: 0.5855 - val_accuracy: 0.8313
Epoch 2/100
24/24 [==============================] - 0s 4ms/step - loss: 0.6105 - accuracy: 0.6447 - val_loss: 0.5099 - val_accuracy: 0.8795
Epoch 3/100
24/24 [==============================] - 0s 4ms/step - loss: 0.5772 - accuracy: 0.7012 - val_loss: 0.4365 - val_accuracy: 0.8675
Epoch 4/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5045 - accuracy: 0.7658 - val_loss: 0.4019 - val_accuracy: 0.8675
Epoch 5/100
24/24 [==============================] - 0s 4ms/step - loss: 0.5077 - accuracy: 0.7779 - val_loss: 0.3762 - val_accuracy: 0.8795
Epoch 6/100
24/24 [==============================] - 0s 4ms/step - loss: 0.4918 - accuracy: 0.7699 - val_loss: 0.3610 - val_accuracy: 0.8795
Epoch 7/100
24/24 [==============================] - 0s 4ms/step - loss: 0.4725 - accuracy: 0.7954 - val_loss: 0.3516 - val_accuracy: 0.8795
Epoch 8/100
